In [1]:
import numpy as np
import pandas as pd
import re
import ast
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.util import bigrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lijhu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lijhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lijhu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lijhu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lijhu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Merge the two dataframes

In [2]:
anime_df = pd.read_csv('data/anime_df.csv')
anime_df.head()

,id,title,picture_url
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...
3,9253,Steins;Gate,https://api-cdn.myanimelist.net/images/anime/1...
4,28977,Gintama°,https://api-cdn.myanimelist.net/images/anime/3...


In [3]:
anime_details_df = pd.read_csv('data/anime_details_df.csv')
anime_details_df.head()

,id,alternative_titles,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,background,studios,statistics
0,51535,{'synonyms': ['Shingeki no Kyojin: The Final S...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,NaN,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':..."
1,5114,{'synonyms': ['Hagane no Renkinjutsushi: Fullm...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,NaN,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':..."
2,41467,{'synonyms': ['Bleach: Thousand-Year Blood War...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,Bleach: Sennen Kessen-hen adapts volume 55 to ...,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ..."
3,9253,"{'synonyms': [], 'en': 'Steins;Gate', 'ja': 'S...",Eccentric scientist Rintarou Okabe has a never...,9.08,4,13,white,tv,finished_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,"{'year': 2011, 'season': 'spring'}",visual_novel,1460,pg_13,Steins;Gate is based on 5pb. and Nitroplus' vi...,"[{'id': 314, 'name': 'White Fox'}]","{'status': {'watching': '157936', 'completed':..."
4,28977,"{'synonyms': [""Gintama' (2015)""], 'en': 'Ginta...","Gintoki, Shinpachi, and Kagura return as the f...",9.07,5,334,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 4, 'name'...",51,"{'year': 2015, 'season': 'spring'}",manga,1440,pg_13,NaN,"[{'id': 1258, 'name': 'Bandai Namco Pictures'}]","{'status': {'watching': '64460', 'completed': ..."


In [4]:
df = anime_df.merge(anime_details_df, on='id', how='inner')
df.head()

,id,title,picture_url,alternative_titles,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,background,studios,statistics
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Shingeki no Kyojin: The Final S...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,NaN,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':..."
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Hagane no Renkinjutsushi: Fullm...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,NaN,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':..."
2,41467,Bleach: Sennen Kessen-hen,https://api-cdn.myanimelist.net/images/anime/1...,{'synonyms': ['Bleach: Thousand-Year Blood War...,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.08,3,495,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",13,"{'year': 2022, 'season': 'fall'}",manga,1471,r,Bleach: Sennen Kessen-hen adapts volume 55 to ...,"[{'id': 1, 'name': 'Pierrot'}]","{'status': {'watching': '95738', 'completed': ..."
3,9253,Steins;Gate,https://api-cdn.myanimelist.net/images/anime/1...,"{'synonyms': [], 'en': 'Steins;Gate', 'ja': 'S...",Eccentric scientist Rintarou Okabe has a never...,9.08,4,13,white,tv,finished_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,"{'year': 2011, 'season': 'spring'}",visual_novel,1460,pg_13,Steins;Gate is based on 5pb. and Nitroplus' vi...,"[{'id': 314, 'name': 'White Fox'}]","{'status': {'watching': '157936', 'completed':..."
4,28977,Gintama°,https://api-cdn.myanimelist.net/images/anime/3...,"{'synonyms': [""Gintama' (2015)""], 'en': 'Ginta...","Gintoki, Shinpachi, and Kagura return as the f...",9.07,5,334,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 4, 'name'...",51,"{'year': 2015, 'season': 'spring'}",manga,1440,pg_13,NaN,"[{'id': 1258, 'name': 'Bandai Namco Pictures'}]","{'status': {'watching': '64460', 'completed': ..."


# EDA

In [5]:
df.shape

(10000, 20)

In [6]:
# check duplicated rows
df['id'].duplicated().value_counts()

False    10000
Name: id, dtype: int64

In [7]:
# check missing values
df.isnull().sum()

id                             0
title                          0
picture_url                    0
alternative_titles             0
synopsis                     180
mean                           0
rank                           0
popularity                     0
nsfw                           0
media_type                     0
status                         0
genres                        17
num_episodes                   0
start_season                 139
source                       975
average_episode_duration       0
rating                        58
background                  8304
studios                        0
statistics                     0
dtype: int64

# Data Cleaning

In [8]:
# drop rows where the synopsis/plot is null because it's an important feature for content-based-filtering
df.dropna(subset=['synopsis'], inplace=True)
# drop the background column since a lot of data are missing
df.drop('background', axis=1, inplace=True)

In [9]:
df.shape

(9820, 19)

In [10]:
# normalize the alternative_titles column
df['alternative_titles'] = df['alternative_titles'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('alternative_titles')))

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST


In [11]:
# get rid of unnecessary characters in the genres column
df['genres'] = df['genres'].apply(lambda x: re.sub('[\[\]{}\'0-9:,]', '', str(x)))
df['genres'] = df['genres'].apply(lambda x: str(x).replace('id', '').replace('name', ''))

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,start_season,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,Action Drama Gore Military Shou...,2,"{'year': 2023, 'season': 'winter'}",manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,Action Adventure Drama Fantasy ...,64,"{'year': 2009, 'season': 'spring'}",manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST


In [12]:
# normalize the start_season column
df['start_season'] = df['start_season'].replace(np.nan, 'None')
df['start_season'] = df['start_season'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('start_season')))

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja,year,season
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,Action Drama Gore Military Shou...,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,Action Adventure Drama Fantasy ...,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring


In [13]:
# filter out anime that aired before 2000 because I don't want to recommend animes that's too old.
df = df[df['year'] >= 2000]
df.shape

(7705, 22)

In [14]:
# get rid of unnecessary characters in the studios column
df['studios'] = df['studios'].apply(lambda x: re.sub('[\[\]{}\'0-9:,]', '', str(x)))
df['studios'] = df['studios'].apply(lambda x: str(x).replace('id', '').replace('name', ''))

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,statistics,synonyms,en,ja,year,season
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,Action Drama Gore Military Shou...,2,manga,3690,r,MAPPA,"{'status': {'watching': '188176', 'completed':...","[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,Action Adventure Drama Fantasy ...,64,manga,1460,r,Bones,"{'status': {'watching': '241806', 'completed':...",[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring


In [15]:
# normalize the statistics column
df['statistics'] = df['statistics'].apply(lambda x: ast.literal_eval(x))
df = df.join(pd.json_normalize(df.pop('statistics')))

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,synonyms,en,ja,year,season,num_list_users,status.watching,status.completed,status.on_hold,status.dropped,status.plan_to_watch
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,Action Drama Gore Military Shou...,2,manga,3690,r,MAPPA,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,382102.0,188176,724,18741,1009,173452
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,Action Adventure Drama Fantasy ...,64,manga,1460,r,Bones,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,3126275.0,241806,2260153,106496,52671,465149


In [16]:
df.columns

Index(['id', 'title', 'picture_url', 'synopsis', 'mean', 'rank', 'popularity',
       'nsfw', 'media_type', 'status', 'genres', 'num_episodes', 'source',
       'average_episode_duration', 'rating', 'studios', 'synonyms', 'en', 'ja',
       'year', 'season', 'num_list_users', 'status.watching',
       'status.completed', 'status.on_hold', 'status.dropped',
       'status.plan_to_watch'],
      dtype='object')

In [17]:
# rename columns
df = df.rename(columns={'synonyms':'alternative_titles', 'en':'english_title', 'ja':'japanese_title',
                        'status.watching':'num_of_ppl_watching','status.completed':'num_of_ppl_completed', 
                        'status.on_hold':'num_of_ppl_on_hold', 'status.dropped':'num_of_ppl_dropped',
                        'status.plan_to_watch':'num_of_ppl_plan_to_watch'})

df.head(2)

,id,title,picture_url,synopsis,mean,rank,popularity,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,alternative_titles,english_title,japanese_title,year,season,num_list_users,num_of_ppl_watching,num_of_ppl_completed,num_of_ppl_on_hold,num_of_ppl_dropped,num_of_ppl_plan_to_watch
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,https://api-cdn.myanimelist.net/images/anime/1...,The conclusion to Shingeki no Kyojin.,9.11,1,540,white,special,currently_airing,Action Drama Gore Military Shou...,2,manga,3690,r,MAPPA,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,382102.0,188176,724,18741,1009,173452
1,5114,Fullmetal Alchemist: Brotherhood,https://api-cdn.myanimelist.net/images/anime/1...,After a horrific alchemy experiment goes wrong...,9.11,2,3,white,tv,finished_airing,Action Adventure Drama Fantasy ...,64,manga,1460,r,Bones,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,3126275.0,241806,2260153,106496,52671,465149


# Text Preprocessing

In [18]:
# combine features
features = ['title', 'synopsis', 'media_type', 'status', 'genres', 'source', 'rating', 'studios', 'alternative_titles',
            'english_title', 'japanese_title', 'season']

df['summary'] = df[features].apply(lambda x: ', '.join(x.astype(str)), axis=1)

df['summary'][1]

'Fullmetal Alchemist: Brotherhood, After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse\'s body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse\'s soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher\'s Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exc

In [19]:
# apply lower case and tokenize
df['tokens'] = df['summary'].apply(str.lower).apply(word_tokenize)

print(df['tokens'][1])

['fullmetal', 'alchemist', ':', 'brotherhood', ',', 'after', 'a', 'horrific', 'alchemy', 'experiment', 'goes', 'wrong', 'in', 'the', 'elric', 'household', ',', 'brothers', 'edward', 'and', 'alphonse', 'are', 'left', 'in', 'a', 'catastrophic', 'new', 'reality', '.', 'ignoring', 'the', 'alchemical', 'principle', 'banning', 'human', 'transmutation', ',', 'the', 'boys', 'attempted', 'to', 'bring', 'their', 'recently', 'deceased', 'mother', 'back', 'to', 'life', '.', 'instead', ',', 'they', 'suffered', 'brutal', 'personal', 'loss', ':', 'alphonse', "'s", 'body', 'disintegrated', 'while', 'edward', 'lost', 'a', 'leg', 'and', 'then', 'sacrificed', 'an', 'arm', 'to', 'keep', 'alphonse', "'s", 'soul', 'in', 'the', 'physical', 'realm', 'by', 'binding', 'it', 'to', 'a', 'hulking', 'suit', 'of', 'armor', '.', 'the', 'brothers', 'are', 'rescued', 'by', 'their', 'neighbor', 'pinako', 'rockbell', 'and', 'her', 'granddaughter', 'winry', '.', 'known', 'as', 'a', 'bio-mechanical', 'engineering', 'prodig

In [20]:
# remove stop words and keep letters only
stop_words = stopwords.words('english')
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words and word.isalpha()])

print(df['tokens'][1])

['fullmetal', 'alchemist', 'brotherhood', 'horrific', 'alchemy', 'experiment', 'goes', 'wrong', 'elric', 'household', 'brothers', 'edward', 'alphonse', 'left', 'catastrophic', 'new', 'reality', 'ignoring', 'alchemical', 'principle', 'banning', 'human', 'transmutation', 'boys', 'attempted', 'bring', 'recently', 'deceased', 'mother', 'back', 'life', 'instead', 'suffered', 'brutal', 'personal', 'loss', 'alphonse', 'body', 'disintegrated', 'edward', 'lost', 'leg', 'sacrificed', 'arm', 'keep', 'alphonse', 'soul', 'physical', 'realm', 'binding', 'hulking', 'suit', 'armor', 'brothers', 'rescued', 'neighbor', 'pinako', 'rockbell', 'granddaughter', 'winry', 'known', 'engineering', 'prodigy', 'winry', 'creates', 'prosthetic', 'limbs', 'edward', 'utilizing', 'automail', 'tough', 'versatile', 'metal', 'used', 'robots', 'combat', 'armor', 'years', 'training', 'elric', 'brothers', 'set', 'quest', 'restore', 'bodies', 'locating', 'philosopher', 'powerful', 'gem', 'allows', 'alchemist', 'defy', 'tradi

In [21]:
#  apply tag to each token
df['tokens'] = df['tokens'].apply(lambda x: nltk.pos_tag(x))
print(df['tokens'][1])

[('fullmetal', 'JJ'), ('alchemist', 'NN'), ('brotherhood', 'NN'), ('horrific', 'NN'), ('alchemy', 'NN'), ('experiment', 'NN'), ('goes', 'VBZ'), ('wrong', 'JJ'), ('elric', 'JJ'), ('household', 'NN'), ('brothers', 'NNS'), ('edward', 'VBP'), ('alphonse', 'NN'), ('left', 'VBD'), ('catastrophic', 'JJ'), ('new', 'JJ'), ('reality', 'NN'), ('ignoring', 'VBG'), ('alchemical', 'JJ'), ('principle', 'NN'), ('banning', 'VBG'), ('human', 'JJ'), ('transmutation', 'NN'), ('boys', 'NNS'), ('attempted', 'VBD'), ('bring', 'VBG'), ('recently', 'RB'), ('deceased', 'VBN'), ('mother', 'NN'), ('back', 'RB'), ('life', 'NN'), ('instead', 'RB'), ('suffered', 'VBD'), ('brutal', 'JJ'), ('personal', 'JJ'), ('loss', 'NN'), ('alphonse', 'NN'), ('body', 'NN'), ('disintegrated', 'VBD'), ('edward', 'RB'), ('lost', 'VBN'), ('leg', 'NN'), ('sacrificed', 'VBN'), ('arm', 'JJ'), ('keep', 'NN'), ('alphonse', 'NN'), ('soul', 'NN'), ('physical', 'JJ'), ('realm', 'NN'), ('binding', 'VBG'), ('hulking', 'VBG'), ('suit', 'NN'), ('a

In [22]:
# convert tags to the tags that WordNetLemmatizer uses
def convert_pos(pos):
    if pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# lemmatize words with pos-tags
df['cleaned_tokens'] = df['tokens'].apply(lambda x: [WordNetLemmatizer().lemmatize(w[0], pos=convert_pos(w[1])) for w in x])
print(df['cleaned_tokens'][1])

['fullmetal', 'alchemist', 'brotherhood', 'horrific', 'alchemy', 'experiment', 'go', 'wrong', 'elric', 'household', 'brother', 'edward', 'alphonse', 'leave', 'catastrophic', 'new', 'reality', 'ignore', 'alchemical', 'principle', 'ban', 'human', 'transmutation', 'boy', 'attempt', 'bring', 'recently', 'decease', 'mother', 'back', 'life', 'instead', 'suffer', 'brutal', 'personal', 'loss', 'alphonse', 'body', 'disintegrate', 'edward', 'lose', 'leg', 'sacrifice', 'arm', 'keep', 'alphonse', 'soul', 'physical', 'realm', 'bind', 'hulk', 'suit', 'armor', 'brother', 'rescue', 'neighbor', 'pinako', 'rockbell', 'granddaughter', 'winry', 'know', 'engineering', 'prodigy', 'winry', 'create', 'prosthetic', 'limb', 'edward', 'utilize', 'automail', 'tough', 'versatile', 'metal', 'use', 'robot', 'combat', 'armor', 'year', 'train', 'elric', 'brother', 'set', 'quest', 'restore', 'body', 'locate', 'philosopher', 'powerful', 'gem', 'allow', 'alchemist', 'defy', 'traditional', 'law', 'equivalent', 'exchange',

# Generate Vectors using TF-IDF

In [23]:
# combine all tokens into one string
df['cleaned_string'] = df['cleaned_tokens'].apply(lambda x: ' '.join(x))

# transform string to TD-IDF vector and ignore words that appear more than 80% of the time in a document (row)
tf_vec = TfidfVectorizer(ngram_range=(1,2), max_df=0.8)
tfidf_matrix = tf_vec.fit_transform(df['cleaned_string'])
tfidf_array = tfidf_matrix.toarray()

In [24]:
# calculate cos similarity
cos_sim = cosine_similarity(tfidf_array, tfidf_array)
print(cos_sim)

[[1.         0.00815515 0.0018486  ... 0.00257801 0.00134707 0.        ]
 [0.00815515 1.         0.01384911 ... 0.00159783 0.         0.0015851 ]
 [0.0018486  0.01384911 1.         ... 0.0017206  0.         0.00603386]
 ...
 [0.00257801 0.00159783 0.0017206  ... 1.         0.00249598 0.00330105]
 [0.00134707 0.         0.         ... 0.00249598 1.         0.003298  ]
 [0.         0.0015851  0.00603386 ... 0.00330105 0.003298   1.        ]]


# Build Recommendation Engine Based on Similarity Score

In [25]:
# create indices for movies
indices = pd.Series(df.index, index=df['title'])
print(indices)

title
Shingeki no Kyojin: The Final Season - Kanketsu-hen       0
Fullmetal Alchemist: Brotherhood                          1
Bleach: Sennen Kessen-hen                                 2
Steins;Gate                                               3
Gintama°                                                  4
                                                       ... 
Hello Kitty no Shirayuki-hime (2001)                   9814
Hitotsubu ni Kawaranu Ai wo Komete                     9815
Kamigami no Ki                                         9817
Kaze no Toori Michi                                    9818
Kemono to Chat                                         9819
Length: 7705, dtype: int64


In [28]:
# recommendation engine based on cosine similarity
def get_recommendations(title, cosine_sim, indices):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_20 = sim_scores[1:20]
    
    movie_indices = [i[0] for i in top_20]
    
    # Return the top 20 most similar movies
    result = df['title'].iloc[movie_indices]
    
    return result

In [31]:
# example
get_recommendations('Lycoris Recoil', cos_sim, indices)

179                                         Chihayafuru 2
231                                  Dr. Stone: New World
2210    Takanashi Rikka Kai: Chuunibyou demo Koi ga Sh...
137                                         Chihayafuru 3
4036     Chihayafuru 2: Waga Miyo ni Furu Nagame Shima ni
1886                      Suzumiya Haruhi-chan no Yuuutsu
527                                           Dennou Coil
8544                                        Magical Canan
9425                                             Earthian
7194                                   Digimon Adventure:
2708                  Baka to Test to Shoukanjuu Specials
615                                                ReLIFE
5408                           Mayonaka no Occult Koumuin
3828                                        Trinity Seven
1239    Detective Conan Movie 21: The Crimson Love Letter
5130                       Chihayafuru 3: Ima Hitotabi no
2958                                 Digimon Adventure 02
232     Fate/s